In [1]:
import seaborn as sns
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, f1_score, log_loss

In [2]:
param_array = pd.read_csv("parameters/parameter_array.csv").set_index("PARAM")

In [3]:
param_array

,AIRPORTS,LOOKAHEAD,DATA,CONFIG_SUPPORT,EPOCHS,NUMBER_TRIALS,PATIENCE
PARAM,,,,,,,
1,katl,1,PD,0,400,1,10
2,kclt,1,PD,0,400,1,10
3,kden,1,PD,0,400,1,10
4,kdfw,1,PD,0,400,1,10
5,kjfk,1,PD,0,400,1,10
...,...,...,...,...,...,...,...
716,kmem,12,PD_SM,1,400,1,10
717,kmia,12,PD_SM,1,400,1,10
718,kord,12,PD_SM,1,400,1,10


In [4]:
Param_Nums = []
DFs = []

Not_Finishes = []
for param in param_array.index:
    
    try:
        df = pd.read_csv(f"Results/Result_{param}/results_df.csv")
        Param_Nums.append(param)
        DFs.append(df)
        
        
    except:
        
        Not_Finishes.append(param)

    
df = pd.concat(DFs)
df["PARAM"] = Param_Nums
df = pd.merge(df, param_array, on="PARAM")

In [5]:
np.unique(df.LOOKAHEAD)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [6]:
df.columns

Index(['Unnamed: 0', 'param', 'train_loss', 'val_loss', 'test_loss',
       'train_accuracy', 'val_accuracy', 'test_accuracy', 'train_f1', 'val_f1',
       'test_f1', 'PARAM', 'AIRPORTS', 'LOOKAHEAD', 'DATA', 'CONFIG_SUPPORT',
       'EPOCHS', 'NUMBER_TRIALS', 'PATIENCE'],
      dtype='object')

In [7]:
df.groupby(["LOOKAHEAD","CONFIG_SUPPORT","DATA"]).mean()["test_loss"]

LOOKAHEAD  CONFIG_SUPPORT  DATA 
1          0               PD       0.036582
                           PD_SM         NaN
                           SM       0.058301
           1               PD       0.031179
                           PD_SM    0.035943
                                      ...   
12         0               PD_SM    0.121327
                           SM       0.120718
           1               PD       0.109755
                           PD_SM    0.135571
                           SM       0.132294
Name: test_loss, Length: 72, dtype: float64

In [8]:
df.groupby(["CONFIG_SUPPORT","DATA"]).mean()["test_loss"]

CONFIG_SUPPORT  DATA 
0               PD       0.084319
                PD_SM         NaN
                SM       0.115074
1               PD       0.087192
                PD_SM    0.105270
                SM       0.101103
Name: test_loss, dtype: float64

In [9]:
param_array.loc[Not_Finishes]

,AIRPORTS,LOOKAHEAD,DATA,CONFIG_SUPPORT,EPOCHS,NUMBER_TRIALS,PATIENCE
PARAM,,,,,,,
547,kmia,7,SM,1,400,1,10
582,kclt,11,SM,1,400,1,10
616,kmem,2,PD_SM,1,400,1,10
638,kord,4,PD_SM,1,400,1,10
654,kdfw,6,PD_SM,1,400,1,10
673,kden,8,PD_SM,1,400,1,10
696,kmem,10,PD_SM,1,400,1,10
713,kden,12,PD_SM,1,400,1,10
